## VOC에 대한 담당 부서 확인

- RAG 검색을 통해 유사한 VOC에 배정된 "담당 부서"를 확인 후 매칭

#### [중요!] 사전에 Bedrock KB (Knowledge Base) 설정은 완료 되어 있다고 가정 합니다.

In [1]:
# 환경 설정
!pip install boto3 -U
!pip install pandas -U
!pip install openpyxl -U

### Amazon Bedrock Client 설정

In [2]:
import boto3
import pprint
from botocore.client import Config
import json

pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = "ap-northeast-2") ## Claude 리전
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config, region_name = "us-east-1") ## KB 리전


### 미리 생성해 둔 KB 목록 확인
- 사용자 Bedrock 환경에 미리 만들어진 KB의 ID를 확인

In [3]:
import boto3

def list_knowledge_bases():
    # Bedrock 클라이언트 생성
    bedrock_agent = boto3.client('bedrock-agent')

    # Knowledge Base 목록 가져오기
    response = bedrock_agent.list_knowledge_bases(
        maxResults=100  # 한 번에 가져올 최대 결과 수
    )

    # 결과 출력
    if 'knowledgeBaseSummaries' in response:
        print("Found Knowledge Bases:")
        for kb in response['knowledgeBaseSummaries']:
            print(f"- Name: {kb['name']}")
            print(f"  ID: {kb['knowledgeBaseId']}")
            print(f"  Description: {kb.get('description', 'N/A')}")
            print(f"  Status: {kb['status']}")
            print(f"  Last Updated: {kb['updatedAt']}")
            print("---")
    else:
        print("No Knowledge Bases found.")

    # 페이지네이션 처리
    while 'nextToken' in response:
        response = bedrock_agent.list_knowledge_bases(
            maxResults=100,
            nextToken=response['nextToken']
        )
        
        for kb in response['knowledgeBaseSummaries']:
            print(f"- Name: {kb['name']}")
            print(f"  ID: {kb['knowledgeBaseId']}")
            print(f"  Description: {kb.get('description', 'N/A')}")
            print(f"  Status: {kb['status']}")
            print(f"  Last Updated: {kb['updatedAt']}")
            print("---")

if __name__ == "__main__":
    list_knowledge_bases()

Found Knowledge Bases:
- Name: 240904-kb-test
  ID: XZARIZ0GMG
  Description: Prompt Flow Builder Workshop
  Status: ACTIVE
  Last Updated: 2024-09-04 00:56:49.549374+00:00
---
- Name: 2024-10-sct-resort-poc-basic
  ID: UEDOZQC6HM
  Description: N/A
  Status: ACTIVE
  Last Updated: 2024-10-21 10:18:05.458218+00:00
---
- Name: 241028-mendix-test
  ID: LFKVALWYSR
  Description: N/A
  Status: ACTIVE
  Last Updated: 2024-10-17 01:09:59.799360+00:00
---
- Name: evaluation-ws-knowledge-base-FAQ_WS_833
  ID: IWDGO2QYUY
  Description: FAQ KB
  Status: ACTIVE
  Last Updated: 2024-09-05 04:26:03.406068+00:00
---
- Name: knowledge-base-quick-start-en
  ID: DJLWVWNGBT
  Description: N/A
  Status: DELETE_UNSUCCESSFUL
  Last Updated: 2024-09-05 05:06:01.616497+00:00
---
- Name: 2024-10-sct-resort-poc
  ID: ACSN5MRWK2
  Description: N/A
  Status: ACTIVE
  Last Updated: 2024-10-21 09:49:07.169355+00:00
---


### KB 설정
- Hybrid Search 사용
- 위에서 KB ID 확인 후, KB ID 변경해야 함

In [4]:
# 사용할 KB ID로 변경 합니다.

kb_id = "ACSN5MRWK2"

In [5]:
# numberOfResults 는 RAG 검색에서 가져올 청크 갯수 입니다.
def retrieve(query, kbId, numberOfResults=10):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults,
                'overrideSearchType': "HYBRID", # 하이브리드 검색
            }
        }
    )

# fetch context from the response
def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults: 
        contexts.append(retrievedResult['content']['text'])
    return contexts


In [6]:
# 입력하는 VOC 
query = """
안녕하세요. 동물원 체험 프로그램에 대해 문의드립니다. 지난번 공지사항에 매주 토요일마다 동물 먹이주기 체험을 진행한다고 하셨는데, 모든 동물에게 해당되는 건가요? 기린이랑 코끼리만 먹이주기 체험을 하고 다른 동물들은 제외된다면 좀 아쉬울 것 같아요. 공지사항만 봐서는 모든 동물이 포함되는 것처럼 보였는데, 혹시 잘못 이해한 건 아닌지 확인 차 여쭤봅니다.
모든 동물들에게 골고루 관심을 가져주셨으면 좋겠어요. 요즘 SNS에서 기린과 코끼리 위주로 홍보하시는 걸 보고 다른 동물 팬들이 많이 아쉬워하더라고요. 저도 그동안 다른 동물 팬들의 요청으로 민원을 자주 넣었지만, 이제는 그냥 구경만 하고 있어요.
하지만 계속 특정 동물들만 홍보하신다면 팬들이 계속해서 불만을 제기할 것 같아요. 그래도 지난번 '펭귄의 날' 행사는 정말 좋았습니다. 앞으로도 다양한 동물들을 위한 행사 많이 해주세요. 늘 동물원 소식 전해주셔서 감사합니다. 
"""


response = retrieve(query, kb_id, 10)
retrievalResults = response['retrievalResults']
contexts = get_contexts(retrievalResults)

In [7]:
# KB 검색 프롬프트 템플릿 정의

prompt = f"""
Human: 당신은 리조트 회사의 고객 서비스를 전문으로 하는 AI 어시스턴트입니다. 새로운 VOC(Voice of Customer)와 유사한 항목을 찾고, 관련 정보와 담당 부서를 제공하는 것이 당신의 임무입니다.
다음의 맥락 정보를 사용하여 <question> 태그 안에 있는 질문에 답하세요. 맥락에는 이전 VOC 항목들의 "질문내용", "답변내용", "담당 부서" 등의 정보가 포함되어 있습니다.

<context>
{contexts}
</context>

<question>
{query}
</question>

다음 형식으로 응답해 주세요:
이러한 유형의 문의나 이슈를 처리하는 담당 부서 이름만 답변에 출력하세요.
추가 설명이나 주석 없이 분류 단어만 작성해주세요.
    
예시 응답:
리조트 주토피아운영그룹 동물보전1

Assistant:"""

In [13]:
%%time
import json
import boto3
from botocore.config import Config

# Cross-region inference를 위한 Bedrock 클라이언트 설정
bedrock_runtime = boto3.client(
    'bedrock-runtime',
    region_name='ap-northeast-2',
    config=Config(
        retries=dict(
            max_attempts=3,
            mode='standard'
        )
    )
)

# 프롬프트 템플릿 포맷팅
formatted_prompt = prompt.format(contexts=contexts, query=query)

# Cross-Region (us-east-1 & us-west-2) inference 를 위한 modelID 설정 
#region_name = 'us-east-1'
#account_id = boto3.client('sts').get_caller_identity()['Account']
#inference_profile_id = "us.anthropic.claude-3-5-sonnet-20240620-v1:0" # Claude 3.5 Sonnet v1 (Cross-region inference)
#inference_profile_arn = f"arn:aws:bedrock:{region_name}:{account_id}:inference-profile/{inference_profile_id}"
#print(inference_profile_arn)


try:
    # converse() 메서드를 위한 파라미터 설정
    response = bedrock_runtime.converse(
        #modelId = inference_profile_arn,  # Claude 3.5 Sonnet (Cross-region inference)
        #modelId = 'anthropic.claude-3-sonnet-20240229-v1:0',  # Claude 3 Sonnet
        modelId = 'anthropic.claude-3-5-sonnet-20240620-v1:0',  # Claude 3.5 Sonnet 
        messages=[
            {
                "role": "user",
                "content": [{"text": formatted_prompt}]
            }
        ],
        inferenceConfig={
            "maxTokens": 4096,
            "temperature": 0.0,
            "topP": 0
        }
    )

    # 응답 처리
    if 'output' in response and 'message' in response['output']:
        response_text = response['output']['message']['content'][0]['text']
        print(response_text)
    else:
        print("응답 생성에 실패했습니다.")

except Exception as e:
    print(f"오류 발생: {str(e)}")

리조트 주토피아운영그룹 운영지원
CPU times: user 29.7 ms, sys: 7.49 ms, total: 37.2 ms
Wall time: 2.89 s
